In [1]:
%reload_ext autoreload
%autoreload 2

import sys
sys.path.append('/home/rishibalakrishnan/DECAF/tests')

from typing import Tuple

import networkx as nx
import numpy as np
import pandas as pd
import pytest
import pytorch_lightning as pl
import torch
from sklearn.metrics import precision_score, recall_score, accuracy_score, roc_auc_score
from utils import gen_data_nonlinear, load_adult
from xgboost import XGBClassifier
import random

from decaf import DECAF, DataModule
import pandas as pd
from opacus import PrivacyEngine
from tqdm.notebook import tqdm
from opacus.validators import ModuleValidator
from opacus.data_loader import shape_safe

torch.manual_seed(0)
random.seed(0)
np.random.seed(0)


df, dag_seed, feat_num = load_adult()
X, y = df[:, :-1], df[:, -1]
# v_prop = .2
# v_len = int(len(total)*v_prop)
# np.random.shuffle(total)
# val, train = total[:v_len], total[v_len:]
# X, y = train[:, :-1], train[:, -1]
# X, y = total[:, :-1], total[:, -1]

baseline_clf = XGBClassifier(use_label_encoder=False).fit(X, y)
y_pred = baseline_clf.predict(X)

print(
    "baseline scores",
    precision_score(y, y_pred),
    recall_score(y, y_pred),
    roc_auc_score(y, y_pred),
)

/home/rishibalakrishnan/.local/lib/python3.7/site-packages/requests/__init__.py:104: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (2.3.0)/charset_normalizer (2.0.10) doesn't match a supported version!
  RequestsDependencyWarning)


[02:35:14] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
baseline scores 0.8943365764540936 0.9291957270239252 0.7989745284027456


In [9]:
import decaf
import importlib
importlib.reload(decaf)
from decaf import DECAF, DataModule

In [13]:
#Total list of changes that I've made
#Using y in the model too
#Giving seed for DAG

total, dag_seed, feat_num = load_adult()
v_prop = .2
v_len = int(len(total)*v_prop)
np.random.shuffle(total)
val, train = total[:v_len], total[v_len:]
X, y = train[:, :-1], train[:, -1]

dm = DataModule(train, batch_size=64, val_data=val, num_workers=0)

model = DECAF(
    len(feat_num),
    use_mask=True,
    dag_seed=dag_seed,
    grad_dag_loss=False,
    lambda_privacy=0,
    lambda_gp=10,
    weight_decay=1e-2,
    l1_g=0,
    p_gen=-1,
    d_updates=10,
    batch_size=100,
    feat_num=feat_num
)

trainer = pl.Trainer(max_epochs=50, logger=False, gpus=1)
trainer.fit(model, dm)

/home/rishibalakrishnan/.local/lib/python3.7/site-packages/pytorch_lightning/core/datamodule.py:175: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]

  | Name          | Type             | Params
---------------------------------------------------
0 | generator     | Generator_causal | 289 K 
1 | discriminator | Discriminator    | 57.6 K
---------------------------------------------------
346 K     Trainable params
1.1 K     Non-trainable params
347 K     Total params
1.390     Total estimated model params size (MB)


Initialised adjacency matrix as parsed:
 Parameter containing:
tensor([[0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 1.],
        [0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])


/home/rishibalakrishnan/.local/lib/python3.7/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:617: UserWarning: Checkpoint directory /srv/home/rishibalakrishnan/DECAF/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")


Validation sanity check: 0it [00:00, ?it/s]

/home/rishibalakrishnan/.local/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:111: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 56 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [21]:
#Total list of changes that I've made
#Using y in the model too
#Giving seed for DAG
#d updates at 10 instead of 5

X, y = load_adult()
# v_prop = .2
# v_len = int(len(total)*v_prop)
# np.random.shuffle(total)
# val, train = total[:v_len], total[v_len:]
# X, y = train[:, :-1], train[:, -1]

dm = DataModule(X, num_workers=56)

model = DECAF(
    dm.dims[0],
    use_mask=True,
    grad_dag_loss=False,
    lambda_privacy=0,
    lambda_gp=10,
    weight_decay=1e-2,
    l1_g=0,
    p_gen=-1,
    batch_size=100,
)
trainer = pl.Trainer(max_epochs=10, logger=False)
trainer.fit(model, dm)

/home/rishibalakrishnan/.local/lib/python3.7/site-packages/pytorch_lightning/core/datamodule.py:170: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")
GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/home/rishibalakrishnan/.local/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:1567: UserWarning: GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`.
  "GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`."
/home/rishibalakrishnan/.local/lib/python3.7/site-packages/pytorch_lightning/trainer/configuration_validator.py:116: UserWarning: You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.
  rank_zero_warn("You passed in a `val_

Training: 0it [00:00, ?it/s]

In [17]:
# total, dag_seed, feat_num = load_adult()
# v_prop = .2
# v_len = int(len(total)*v_prop)
# np.random.shuffle(total)
# val, train = total[:v_len], total[v_len:]
# X, y = train[:, :-1], train[:, -1]

batch_size = 64
sample_rate = batch_size / len(train)
noise_multiplier = .5
max_grad_norm = 2

dm = DataModule(train, val, num_workers=0, batch_size=batch_size, enable_dp=True)

model = DECAF(
    len(feat_num),
    use_mask=True,
    dag_seed=dag_seed,
    grad_dag_loss=False,
    lambda_privacy=0,
    lambda_gp=10,
    weight_decay=1e-2,
    l1_g=1,
    p_gen=-1,
    d_updates=10,
    noise_multiplier=noise_multiplier,
    sample_rate=sample_rate,
    max_grad_norm = max_grad_norm,
    batch_size=batch_size,
    enable_dp=True, 
    feat_num=feat_num
)

trainer = pl.Trainer(max_epochs=50, logger=False, gpus=1)
trainer.fit(model, dm)

/home/rishibalakrishnan/.local/lib/python3.7/site-packages/pytorch_lightning/core/datamodule.py:175: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]

  | Name          | Type             | Params
---------------------------------------------------
0 | generator     | Generator_causal | 289 K 
1 | discriminator | GradSampleModule | 57.6 K
---------------------------------------------------
346 K     Trainable params
1.1 K     Non-trainable params
347 K     Total params
1.390     Total estimated model params size (MB)


Initialised adjacency matrix as parsed:
 Parameter containing:
tensor([[0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 1.],
        [0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])


/home/rishibalakrishnan/.local/lib/python3.7/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:617: UserWarning: Checkpoint directory /srv/home/rishibalakrishnan/DECAF/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")


Validation sanity check: 0it [00:00, ?it/s]

/home/rishibalakrishnan/.local/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:111: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 56 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."
/home/rishibalakrishnan/.local/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:111: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 56 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

/home/rishibalakrishnan/.local/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:685: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [14]:
# print("Generated synthetic data")
device = torch.device('cuda')
model.to(device)
# total_synth = model.generator.sequential(
#     dm.dataset.x.to(device),
#     model.sample_z(dm.dataset.x.shape[0]).type_as(dm.dataset.x).to(device),
#     gen_order=model.get_gen_order(),
#     biased_edges=[],
# ).detach().cpu().numpy()
total_synth = (
    model.gen_synthetic(
        dm.train_dataset.x.to(device),
        gen_order=model.get_gen_order(),
    )
    .detach()
    .cpu()
    .numpy()
)

X_synth, y_synth = total_synth[:, :-1], total_synth[:, -1]

y_synth = np.round(y_synth, 0).astype(np.uint8)

# y_synth = baseline_clf.predict(X_synth) #Using the model fitted on the real data to produce predictions
# #I guess these predictions are all one class?
print(f"Generated Y - fraction 1's: {sum(y_synth)/len(y_synth)}")

y_pred = baseline_clf.predict(X_synth)
# y_pred = synth_clf.predict(X_synth)

print(
    "Trained on real data - predicting on synthetic",
    precision_score(y_synth, y_pred),
    recall_score(y_synth, y_pred),
    roc_auc_score(y_synth, y_pred),
)

#Train model only on synthetic data and then see how it performs to actual
synth_clf = XGBClassifier(use_label_encoder=False).fit(X_synth, y_synth)
y_pred = synth_clf.predict(X)
print(
    "Trained on synthetic data - predicting on real",
    precision_score(y, y_pred),
    recall_score(y, y_pred),
    roc_auc_score(y, y_pred),
)

#Holdout set
prop = .2
length = int(prop*len(X))
X_val, y_val = X[:length], y[:length]
X_train, y_train = X[length:], y[length:]
real_clf = XGBClassifier(use_label_encoder=False).fit(X_train, y_train)

real_pred = real_clf.predict(X_val)
print("Real Data on holdout set:",
    precision_score(y_val, real_pred),
    recall_score(y_val, real_pred),
    roc_auc_score(y_val, real_pred),
)

synth_pred = synth_clf.predict(X_val)
print("Synth Data on holdout set:",
    precision_score(y_val, synth_pred),
    recall_score(y_val, synth_pred),
    roc_auc_score(y_val, synth_pred),
)

Generated Y - fraction 1's: 0.6592623290509739
Trained on real data - predicting on synthetic 0.7617584627415931 0.8572416394267035 0.6692557017371903
[03:32:23] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Trained on synthetic data - predicting on real 0.8829650652071086 0.7208186231244483 0.7161240733083089
[03:32:25] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Real Data on holdout set: 0.8820194384449244 0.9017388904223019 0.7692401850282748
Synth Data on holdout set: 0.8868824531516184 0.7184653601987303 0.7212443176721


In [7]:
#Total list of changes that I've made
#Using y in the model too
#Giving seed for DAG

total, dag_seed, feat_num = load_adult()
v_prop = .2
v_len = int(len(total)*v_prop)
np.random.shuffle(total)
val, train = total[:v_len], total[v_len:]
X, y = train[:, :-1], train[:, -1]

dm = DataModule(train, batch_size=64, valid_data=val, num_workers=56)

model = DECAF(
    len(feat_num),
    use_mask=True,
    dag_seed=dag_seed,
    grad_dag_loss=False,
    lambda_privacy=0,
    lambda_gp=0,
    weight_decay=1e-2,
    l1_g=0,
    p_gen=-1,
    d_updates=10,
    batch_size=100,
    feat_num=feat_num
)

trainer = pl.Trainer(max_epochs=50, logger=False, gpus=1)
trainer.fit(model, dm)

/home/rishibalakrishnan/.local/lib/python3.7/site-packages/pytorch_lightning/core/datamodule.py:175: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]

  | Name          | Type             | Params
---------------------------------------------------
0 | generator     | Generator_causal | 289 K 
1 | discriminator | Discriminator    | 57.6 K
---------------------------------------------------
346 K     Trainable params
1.1 K     Non-trainable params
347 K     Total params
1.390     Total estimated model params size (MB)


Initialised adjacency matrix as parsed:
 Parameter containing:
tensor([[0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 1.],
        [0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])


/home/rishibalakrishnan/.local/lib/python3.7/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:617: UserWarning: Checkpoint directory /srv/home/rishibalakrishnan/DECAF/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [34]:
from torch import nn
h_dim = 200
ml = nn.ModuleList([nn.Sequential(
                nn.Linear(h_dim, h_dim),
                nn.ReLU(inplace=True),
                nn.Linear(h_dim, 2)
            ) 
            for i in range(5)])

In [59]:
total_synth[0]

array([9.99695897e-01, 2.17938083e-16, 1.34072951e-28, 2.73538471e-21,
       5.48444431e-15, 7.62095332e-01, 9.99686360e-01, 9.98649299e-01,
       4.39007199e-05, 3.77879222e-03, 9.79912118e-04, 6.31651492e-04,
       1.31357185e-04, 6.44884247e-04, 1.84138201e-03, 1.17452955e-05,
       3.46159941e-05, 1.24797898e-05, 8.97080824e-03, 1.89770130e-03,
       3.03674133e-05, 9.39239617e-05, 2.42579915e-03, 2.19374849e-03,
       1.04936818e-03, 5.97884785e-03, 3.37162381e-03, 1.76287447e-08,
       3.39009639e-05, 1.57756207e-04, 1.49013812e-03, 1.22399665e-02,
       1.57482827e-05, 1.72681571e-03, 4.76889654e-05, 1.33217103e-03,
       2.01368774e-03, 1.39350025e-03, 1.13616064e-04, 1.14452050e-04,
       6.21793297e-05, 6.14365053e-05, 5.08206249e-05, 3.21408720e-07,
       1.13838381e-04, 6.61843037e-03, 4.20357464e-05, 2.15125736e-04,
       1.00000000e+00, 0.00000000e+00, 8.85730128e-26, 5.63101136e-33,
       0.00000000e+00, 3.82171960e-18, 4.28199126e-15, 5.26398480e-01,
      

In [32]:
batch_size = 64
sampling_rate = batch_size / len(total)
noise_multiplier=1.1
max_grad_norm=2.
delta=1e-5

dm = DataModule(total, num_workers=0, batch_size=batch_size)

device = torch.device('cuda')

model = DECAF(
    dm.dims[0],
    use_mask=True,
    dag_seed=dag_seed,
    grad_dag_loss=False,
    lambda_privacy=0,
    lambda_gp=10,
    weight_decay=1e-2,
    l1_g=0,
    p_gen=-1,
    d_updates=10,
)

dict_opt = model.configure_optimizers()
d_opt = dict_opt[0]["optimizer"]
d_iter = dict_opt[0]["frequency"]

g_opt = dict_opt[1]["optimizer"]
g_iter = dict_opt[1]["frequency"]

# train_dataloader = dm.train_dataloader()

# privacy_engine = PrivacyEngine()

# discriminator, d_opt, train_dataloader = privacy_engine.make_private(module=model.discriminator, 
#                                                             optimizer=d_opt, 
#                                                             data_loader=train_dataloader, 
#                                                             noise_multiplier=noise_multiplier, 
#                                                             max_grad_norm=max_grad_norm) 
# model.discriminator = discriminator

model.cuda()
model.train()
torch.set_grad_enabled(True)

epochs = 50
for i in range(epochs):
    data_bar = tqdm(train_dataloader)
    for idx, batch in enumerate(data_bar):
        opt_idx = int(idx % (g_iter + d_iter) == d_iter)

        batch = batch.cuda()
        d_opt.zero_grad()

        z = model.sample_z(batch.shape[0]).type_as(batch).to(device)

        generated_batch = model.generator.sequential(batch, z, model.get_gen_order())

        if opt_idx == 0:
            fake_loss = torch.mean(model.discriminator(generated_batch.detach()))
            real_loss = torch.mean(model.discriminator(batch))
            gp = model.hparams.lambda_gp * model.compute_gradient_penalty(batch, generated_batch)
            loss = fake_loss - real_loss + gp

            loss.backward()
            d_opt.step()
            d_opt.zero_grad()

        # model.discriminator.remove_hooks()
        # gp = model.hparams.lambda_gp * model.compute_gradient_penalty(batch, generated_batch)
        # gp.backward(create_graph=True, retain_graph=True)
        # model.discriminator.add_hooks()

        #Train discriminator
        # fake_output = model.discriminator(generated_batch.detach())
        # fake_loss = torch.mean(fake_output)
        # fake_loss.backward()
        # d_opt.step()
        # d_opt.zero_grad()

        # real_output = model.discriminator(batch)
        # real_loss = - torch.mean(real_output)
        # real_loss.backward()

        # d_opt.step()
        # d_opt.zero_grad()

       
            disc_loss = fake_loss.detach() - real_loss.detach()
            data_bar.set_description(f"epoch: {i}, Loss: {disc_loss.item():.2f}, GP: {gp.detach().item():.2f}")

        else:
            #Train generator
            g_opt.zero_grad()
            g_output = model.discriminator(generated_batch)
            g_loss = - torch.mean(g_output)
            g_loss.backward()
            g_opt.step()
            data_bar.set_description(f"epoch: {i}, Loss: {g_loss.detach().item():.2f}")
    # eps = privacy_engine.get_epsilon(1e-5)
    # print(f"EPOCH {i} EPSILON {eps}")

# for i in range(epochs):
#     for idx, batch in tqdm(enumerate(train_dataloader)):
#         output = model.training_step(batch, idx, 0)
#         loss = output["loss"]
#         loss.backward()
#         d_opt.step()
#         d_opt.zero_grad()
        

#         # if idx % d_iter == (d_iter - 1):
#         # if idx % 5 == 0:
#         output = model.training_step(batch, idx, 1)
#         loss = output["loss"]
#         loss.backward()
#         g_opt.step()
#         g_opt.zero_grad()

Initialised adjacency matrix as parsed:
 Parameter containing:
tensor([[0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 1.],
        [0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])


/home/rishibalakrishnan/.local/lib/python3.7/site-packages/pytorch_lightning/core/datamodule.py:175: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")
/home/rishibalakrishnan/.local/lib/python3.7/site-packages/pytorch_lightning/core/datamodule.py:170: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")


  0%|          | 0/472 [00:00<?, ?it/s]

  0%|          | 0/472 [00:00<?, ?it/s]

  0%|          | 0/472 [00:00<?, ?it/s]

...

  0%|          | 0/472 [00:00<?, ?it/s]

  0%|          | 0/472 [00:00<?, ?it/s]

  0%|          | 0/472 [00:00<?, ?it/s]

  0%|          | 0/472 [00:00<?, ?it/s]

  0%|          | 0/472 [00:00<?, ?it/s]

In [46]:
import decaf
import importlib
importlib.reload(decaf)
from decaf import DECAF

NameError: name 'model' is not defined

In [8]:
model.generator.M.shape

torch.Size([85, 85])

In [ ]:
synth_data

In [28]:
import decaf
import importlib
importlib.reload(decaf)
from decaf import DECAF, DataModule

In [1]:
batch_size = 64
sample_rate = batch_size / len(train)
noise_multiplier = .5
max_grad_norm = 1.1

dm = DataModule(train, val, num_workers=56, batch_size=batch_size, enable_dp=True)

model = DECAF(
    dm.dims[0],
    use_mask=True,
    dag_seed=dag_seed,
    grad_dag_loss=False,
    lambda_privacy=0,
    lambda_gp=10,
    weight_decay=1e-2,
    l1_g=0,
    p_gen=-1,
    d_updates=10,
    noise_multiplier=noise_multiplier,
    sample_rate=sample_rate,
    max_grad_norm = max_grad_norm,
    batch_size=batch_size,
    enable_dp=True
)

trainer = pl.Trainer(max_epochs=50, logger=False, gpus=1)
trainer.fit(model, dm)

NameError: name 'train' is not defined

In [7]:
synth_data = model.gen_synthetic(torch.ones((30162, model.x_dim)).cuda()).detach().cpu().numpy()
X_synth, y_synth = synth_data[:, :-1], synth_data[:, -1]
y_synth = np.round(y_synth, 0)

clf = XGBClassifier(use_label_encoder=False).fit(X_synth, y_synth)

ValueError: The label must consist of integer labels of form 0, 1, 2, ..., [num_class - 1].

In [6]:
sum(y_synth)/len(y_synth)

1.0

In [5]:

# y_synth = y_synth / np.max(y_synth)

In [11]:
total[0]

array([0.        , 0.53424658, 1.        , 0.        , 2.        ,
       0.86666667, 8.        , 0.40816327, 5.        , 3.        ,
       1.        ])

In [12]:
feat_num = {0:1, 1:5, 2:2, 3:4}

In [15]:
previous_idx = 0
feat_ranges = {}
for idx in sorted(feat_num.keys()):
    feat_ranges[idx] = np.arange(previous_idx, previous_idx + feat_num[idx])
    previous_idx += feat_num[idx]

In [5]:
dag_seed = [[0, 4], [1, 4], [2, 4], [3, 4], [0, 5], [4, 5], [1, 5], [2, 5], [3, 5], [0, 6], [1, 6], [2, 6], [4, 6], [5, 6], [0, 7], [1, 7], [4, 7], [3, 7], [2, 7], [5, 7], [1, 8], [4, 8], [2, 8], [5, 8], [3, 8], [4, 9], [5, 9], [1, 9], [2, 9], [3, 9], [0, 10], [1, 10], [2, 10], [3, 10], [4, 10], [5, 10], [6, 10], [7, 10], [8, 10], [9, 10]]

M_init = torch.rand(11, 11) * 0.0
for pair in dag_seed:
    M_init[pair[0], pair[1]] = 1

In [6]:
feat_num = {0: 5, 1: 1, 2:1, 3: 41, 4: 7, 5: 1, 6:14, 7: 1, 8:7, 9:6, 10:1}
oh_M = torch.empty((0, len(feat_num)))
for idx in sorted(feat_num.keys()):
    oh_idx = torch.unsqueeze(M_init[idx, :], 0)
    oh_idx = torch.tile(oh_idx, (feat_num[idx], 1))
    oh_M = torch.cat([oh_M, oh_idx], 0)

In [10]:
oh_M

tensor([[0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 1.],
        [0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 1.],
        [0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 1.],
        [0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 1.],
        [0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 1.],
        [0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1.],
        [0., 0., 0., 0., 1.,

In [36]:
M_fake = torch.empty((11, 0))
a = torch.tile(torch.unsqueeze(M_init[:, -1], 1), (1, 5))
torch.cat([M_fake, a], 1)

tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0.]])

In [6]:
# print("Generated synthetic data")
device = torch.device('cuda')
model.to(device)
total_synth = (
    model.gen_synthetic(
        dm.dataset.x.to(device),
        gen_order=model.get_gen_order(),
    )
    .detach()
    .cpu()
    .numpy()
)

X_synth, y_synth = total_synth[:, :-1], total_synth[:, -1]

y_synth = np.round(y_synth, 0).astype(np.uint8)

# y_synth = baseline_clf.predict(X_synth) #Using the model fitted on the real data to produce predictions
# #I guess these predictions are all one class?
print(f"Generated Y - fraction 1's: {sum(y_synth)/len(y_synth)}")

y_pred = baseline_clf.predict(X_synth)
# y_pred = synth_clf.predict(X_synth)

print(
    "Trained on real data - predicting on synthetic",
    precision_score(y_synth, y_pred),
    recall_score(y_synth, y_pred),
    roc_auc_score(y_synth, y_pred),
)

#Train model only on synthetic data and then see how it performs to actual
synth_clf = XGBClassifier(use_label_encoder=False).fit(X_synth, y_synth)
y_pred = synth_clf.predict(X)
print(
    "Trained on synthetic data - predicting on real",
    precision_score(y, y_pred),
    recall_score(y, y_pred),
    roc_auc_score(y, y_pred),
)

#Holdout set
prop = .2
length = int(prop*len(X))
X_val, y_val = X[:length], y[:length]
X_train, y_train = X[length:], y[length:]
real_clf = XGBClassifier(use_label_encoder=False).fit(X_train, y_train)

real_pred = real_clf.predict(X_val)
print("Real Data on holdout set:",
    precision_score(y_val, real_pred),
    recall_score(y_val, real_pred),
    roc_auc_score(y_val, real_pred),
)

synth_pred = synth_clf.predict(X_val)
print("Synth Data on holdout set:",
    precision_score(y_val, synth_pred),
    recall_score(y_val, synth_pred),
    roc_auc_score(y_val, synth_pred),
)

Generated Y - fraction 1's: 0.7225979709568331
Trained on real data - predicting on synthetic 0.721552386359841 0.9912365221381051 0.49741101832971946
[08:34:24] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Trained on synthetic data - predicting on real 0.7697882352941177 0.7220799858744592 0.5352541644875759
[08:34:25] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Real Data on holdout set: 0.8761497326203208 0.905393457117595 0.7607206012378426
Synth Data on holdout set: 0.7665250236071766 0.7177276746242264 0.530946065428824


In [41]:
len(total)

30162

In [ ]:
prop = .2
length = int(prop*len(X))
X_val, y_val = X[:length], y[:length]

dataloader = DataModule()



X_train, y_train = X[length:], y[length:]
real_clf = XGBClassifier(use_label_encoder=False).fit(X_train, y_train)

real_pred = real_clf.predict(X_val)
print("Real Data on holdout set:",
    precision_score(y_val, real_pred),
    recall_score(y_val, real_pred),
    roc_auc_score(y_val, real_pred),
)

synth_pred = synth_clf.predict(X_val)
print("Synth Data on holdout set:",
    precision_score(y_val, synth_pred),
    recall_score(y_val, synth_pred),
    roc_auc_score(y_val, synth_pred),
)

In [47]:
#Some EDA
path = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
names = [
    "age",
    "workclass",
    "fnlwgt",
    "education",
    "education-num",
    "marital-status",
    "occupation",
    "relationship",
    "race",
    "sex",
    "capital-gain",
    "capital-loss",
    "hours-per-week",
    "native-country",
    "label",
]
df = pd.read_csv(path, names=names, index_col=False)
df = df.applymap(lambda x: x.strip() if type(x) is str else x)

In [48]:
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,label
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [51]:
a = [
            "Tech-support",
            "Craft-repair",
            "Other-service",
            "Sales",
            "Exec-managerial",
            "Prof-specialty",
            "Handlers-cleaners",
            "Machine-op-inspct",
            "Adm-clerical",
            "Farming-fishing",
            "Transport-moving",
            "Priv-house-serv",
            "Protective-serv",
            "Armed-Forces",
        ]

In [52]:
len(a)

14

In [63]:
path = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
names = [
    "age",
    "workclass",
    "fnlwgt",
    "education",
    "education-num",
    "marital-status",
    "occupation",
    "relationship",
    "race",
    "sex",
    "capital-gain",
    "capital-loss",
    "hours-per-week",
    "native-country",
    "label",
]
df = pd.read_csv(path, names=names, index_col=False)
df = df.applymap(lambda x: x.strip() if type(x) is str else x)

for col in df:
    if df[col].dtype == "object":
        df = df[df[col] != "?"]

replace = [
    [
        "Private",
        "Self-emp-not-inc",
        "Self-emp-inc",
        "Federal-gov",
        "Local-gov",
        "State-gov",
        "Without-pay",
        "Never-worked",
    ],
    [
        "Bachelors",
        "Some-college",
        "11th",
        "HS-grad",
        "Prof-school",
        "Assoc-acdm",
        "Assoc-voc",
        "9th",
        "7th-8th",
        "12th",
        "Masters",
        "1st-4th",
        "10th",
        "Doctorate",
        "5th-6th",
        "Preschool",
    ],
    [
        "Married-civ-spouse",
        "Divorced",
        "Never-married",
        "Separated",
        "Widowed",
        "Married-spouse-absent",
        "Married-AF-spouse",
    ],
    [
        "Tech-support",
        "Craft-repair",
        "Other-service",
        "Sales",
        "Exec-managerial",
        "Prof-specialty",
        "Handlers-cleaners",
        "Machine-op-inspct",
        "Adm-clerical",
        "Farming-fishing",
        "Transport-moving",
        "Priv-house-serv",
        "Protective-serv",
        "Armed-Forces",
    ],
    [
        "Wife",
        "Own-child",
        "Husband",
        "Not-in-family",
        "Other-relative",
        "Unmarried",
    ],
    ["White", "Asian-Pac-Islander", "Amer-Indian-Eskimo", "Other", "Black"],
    ["Female", "Male"],
    [
        "United-States",
        "Cambodia",
        "England",
        "Puerto-Rico",
        "Canada",
        "Germany",
        "Outlying-US(Guam-USVI-etc)",
        "India",
        "Japan",
        "Greece",
        "South",
        "China",
        "Cuba",
        "Iran",
        "Honduras",
        "Philippines",
        "Italy",
        "Poland",
        "Jamaica",
        "Vietnam",
        "Mexico",
        "Portugal",
        "Ireland",
        "France",
        "Dominican-Republic",
        "Laos",
        "Ecuador",
        "Taiwan",
        "Haiti",
        "Columbia",
        "Hungary",
        "Guatemala",
        "Nicaragua",
        "Scotland",
        "Thailand",
        "Yugoslavia",
        "El-Salvador",
        "Trinadad&Tobago",
        "Peru",
        "Hong",
        "Holand-Netherlands",
    ],
    [">50K", "<=50K"],
]

for row in replace:
    df = df.replace(row, range(len(row)))

continuous = ["education-num", "hours-per-week", "age"]
for col in continuous:
    c_range = df[col].max() - df[col].min()
    df[col] = df[col] / c_range

df = df[["race", "age", "sex", "native-country", "marital-status", "education-num", "occupation", "hours-per-week", "workclass", "relationship", "label"]]
# df = pd.get_dummies(df, columns=["race", "native-country", "marital-status", "occupation", "workclass", "relationship"])
# df = df.values

feature_num = {0: 5, 1: 1, 2:1, 3: 41, 4: 7, 5: 1, 6:14, 7: 1, 8:7, 9:6, 10:1}

In [65]:
categorical = ["race", "native-country", "marital-status", "occupation", "workclass", "relationship"]
df = pd.concat([pd.get_dummies(df[col]) if col in categorical else df[col] for col in df.columns], axis=1)

In [73]:
pd.concat(cols, axis=1).columns

Index([               0,                1,                2,                3,
                      4,            'age',            'sex',                0,
                      1,                2,                3,                4,
                      5,                6,                7,                8,
                      9,               10,               11,               12,
                     13,               14,               15,               16,
                     17,               18,               19,               20,
                     21,               22,               23,               24,
                     25,               26,               27,               28,
                     29,               30,               31,               32,
                     33,               34,               35,               36,
                     37,               38,               39,               40,
                      0,                1,          

In [56]:
df.shape

(30162, 85)

In [55]:
sum(feature_num.values())

86